In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('../CSV/19.01.2026/fixtures.csv')

# Convert date columns to datetime format
df['created_at'] = pd.to_datetime(df['created_at'])

print(df.head())
print(df.info())

                                     id                          game_week_id  \
0  00c2569b-cc74-4eca-9b0c-3d03feff2fcf  00c76044-16a0-4acf-a0a5-8de00e9cbc06   
1  03ee0cec-342e-4c89-8590-a70dfaffc42e  3fbb2bf0-cef3-48b2-9d04-ecba8bd4165b   
2  03f17591-9c55-4c93-8131-3ff2a3385c48  e1083685-268d-4376-bbc8-14fe74194d46   
3  0487bc30-fc05-4236-a6a5-eafcaded292b  37bc4415-6386-4c1c-9a8b-453802fa72e1   
4  058951f7-3051-4df0-99ba-a86590b9c725  e1083685-268d-4376-bbc8-14fe74194d46   

   fixture_number  home_team               away_team  home_score  away_score  \
0               3    Burnley                  Fulham         2.0         3.0   
1              10    Everton                West Ham         1.0         1.0   
2               5  Tottenham                 Burnley         3.0         0.0   
3               3    Arsenal  Brighton & Hove Albion         2.0         1.0   
4               9    Man Utd                 Arsenal         0.0         1.0   

                        created_

In [23]:
# Load profiles and predictions CSVs
profiles_df = pd.read_csv('../CSV/19.01.2026/profiles.csv')
predictions_df = pd.read_csv('../CSV/19.01.2026/predictions.csv')

# Convert date columns (using format='mixed' to handle various datetime formats)
predictions_df['created_at'] = pd.to_datetime(predictions_df['created_at'], format='mixed')
predictions_df['updated_at'] = pd.to_datetime(predictions_df['updated_at'], format='mixed')
profiles_df['created_at'] = pd.to_datetime(profiles_df['created_at'], format='mixed') if 'created_at' in profiles_df.columns else None

# Merge predictions with profiles (on user_id -> id)
merged_df = predictions_df.merge(profiles_df, left_on='user_id', right_on='id', suffixes=('_pred', '_profile'))

# Merge with fixtures (on fixture_id -> id)
player_predictions = merged_df.merge(df, left_on='fixture_id', right_on='id', suffixes=('', '_fixture'))

# Sort by player name (A-Z) then by created_at (most recent first)
player_predictions = player_predictions.sort_values(by=['username', 'created_at_pred'], ascending=[True, False]).reset_index(drop=True)
player_predictions.index = player_predictions.index + 1

# Select and reorder columns as requested
player_predictions = player_predictions[['username', 'home_team', 'home_score', 'away_team', 'away_score', 'home_prediction', 'away_prediction', 'created_at_pred']]

# Rename columns for clarity
player_predictions.columns = ['Player', 'Home Team', 'Home Score', 'Away Team', 'Away Score', 'Home Prediction', 'Away Prediction', 'Created At']

print("Player Predictions DataFrame created!")
print(f"Shape: {player_predictions.shape}")
print("\nFirst few rows:")
print(player_predictions.head(10))

Player Predictions DataFrame created!
Shape: (6780, 8)

First few rows:
              Player                Home Team  Home Score         Away Team  \
1   Alan Taylor-Reed        Manchester United         NaN   Manchester City   
2   Alan Taylor-Reed        Nottingham Forest         NaN           Arsenal   
3   Alan Taylor-Reed               Sunderland         NaN    Crystal Palace   
4   Alan Taylor-Reed                  Chelsea         NaN         Brentford   
5   Alan Taylor-Reed              Aston Villa         NaN           Everton   
6   Alan Taylor-Reed        Tottenham Hotspur         NaN   West Ham United   
7   Alan Taylor-Reed                Liverpool         NaN           Burnley   
8   Alan Taylor-Reed  Wolverhampton Wanderers         NaN  Newcastle United   
9   Alan Taylor-Reed             Leeds United         NaN            Fulham   
10  Alan Taylor-Reed   Brighton & Hove Albion         NaN   AFC Bournemouth   

    Away Score  Home Prediction  Away Prediction  \
1     

In [24]:
# Display the dataframe with better formatting
# Styled table with grid, alternating row colors, and better contrast
styled_df = player_predictions.head(20).style.set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#4472C4'), ('color', 'white'), ('border', '1px solid black'), ('font-weight', 'bold')]},
    {'selector': 'td', 'props': [('border', '1px solid black'), ('color', '#1a1a1a'), ('padding', '8px')]},
    {'selector': 'tr:nth-child(even) td', 'props': [('background-color', '#F2F2F2'), ('color', '#1a1a1a')]},
    {'selector': 'tr:nth-child(odd) td', 'props': [('background-color', '#FFFFFF'), ('color', '#1a1a1a')]}
]).format({
    'Home Score': '{:.0f}',
    'Away Score': '{:.0f}',
    'Created At': lambda x: x.strftime('%Y-%m-%d %H:%M') if pd.notna(x) else ''
})

display(styled_df)

,Player,Home Team,Home Score,Away Team,Away Score,Home Prediction,Away Prediction,Created At
1,Alan Taylor-Reed,Manchester United,nan,Manchester City,nan,2,2,2026-01-13 19:48
2,Alan Taylor-Reed,Nottingham Forest,nan,Arsenal,nan,0,3,2026-01-13 19:48
3,Alan Taylor-Reed,Sunderland,nan,Crystal Palace,nan,1,1,2026-01-13 19:48
4,Alan Taylor-Reed,Chelsea,nan,Brentford,nan,2,1,2026-01-13 19:48
5,Alan Taylor-Reed,Aston Villa,nan,Everton,nan,2,0,2026-01-13 19:48
6,Alan Taylor-Reed,Tottenham Hotspur,nan,West Ham United,nan,1,1,2026-01-13 19:48
7,Alan Taylor-Reed,Liverpool,nan,Burnley,nan,3,1,2026-01-13 19:48
8,Alan Taylor-Reed,Wolverhampton Wanderers,nan,Newcastle United,nan,1,2,2026-01-13 19:48
9,Alan Taylor-Reed,Leeds United,nan,Fulham,nan,2,2,2026-01-13 19:48
10,Alan Taylor-Reed,Brighton & Hove Albion,nan,AFC Bournemouth,nan,1,2,2026-01-13 19:48
